In [1]:
import pandas as pd
import polars as pl
import numpy as np
import os, gc
from tqdm.auto import tqdm
from matplotlib import pyplot as plt
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning import (LightningDataModule, LightningModule, Trainer)
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, Timer

import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader


from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor
import lightgbm as lgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

import kaggle_evaluation.jane_street_inference_server

In [2]:
class CONFIG:
    seed = 42
    target_col = "responder_6"
    responder_cols = [f"responder_{idx}" for idx in range(9)]
    lag_cols_rename = { f"responder_{idx}_lag_1" : f"responder_{idx}" for idx in range(9)}
    rename_cols = { f"responder_{idx}_last_lag" : f"responder_{idx}_lag_1" for idx in range(9)}
    xgb_feature_cols =  ['symbol_id','time_id'] \
        + [f"feature_{idx:02d}" for idx in range(79)] \
        + [f"responder_{idx}_mean_lag" for idx in range(9)] \
        + [f"responder_{idx}_std_lag" for idx in range(9)] \
        + [f"responder_{idx}_max_lag" for idx in range(9)] \
        + [f"responder_{idx}_last_lag" for idx in range(9)] \
        + [f"responder_{idx}_chg_lag" for idx in range(9)]
    
    nn_feature_cols =   [f"feature_{idx:02d}" for idx in range(79)] \
        + [f"responder_{idx}_last_lag" for idx in range(9)]
        #+ [f"responder_{idx}_mean_lag" for idx in range(9)] \
        #+ [f"responder_{idx}_std_lag" for idx in range(9)] \
        #+ [f"responder_{idx}_max_lag" for idx in range(9)] \
        #+ [f"responder_{idx}_last_lag" for idx in range(9)] \
        #+ [f"responder_{idx}_chg_lag" for idx in range(9)]
    
    model_paths = [
        "/kaggle/input/nn-model-3",
        "/kaggle/input/xgb-model-3/result_900.pkl",
    ]

In [3]:
xgb_model = None
model_path = CONFIG.model_paths[1]
with open( model_path, "rb") as fp:
    result = pickle.load(fp)
    xgb_model = result["model"]

# Show model
display(xgb_model)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device='cuda', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=250, n_gpus=4, n_jobs=None,
             num_parallel_tree=None, ...)

In [ ]:
# Custom R2 metric for validation
def r2_val(y_true, y_pred, sample_weight):
    r2 = 1 - np.average((y_pred - y_true) ** 2, weights=sample_weight) / (np.average((y_true) ** 2, weights=sample_weight) + 1e-38)
    return r2

#LightningModule 是一个神经网络类，标准化了模型训练、验证和优化的流程
'''
神经网络结构如下：
Input(0)→BN(0)→Linear(0→1)→BN(1)→SiLU(1)→Dropout(1)→Linear(1→2)→...→Linear→Tanh→Output
循环中，除了第0层外，都加入SiLU层，所有dropouts列表长度内，都加入Dropout层
'''
class NN(LightningModule):
    def __init__(self, input_dim, hidden_dims, dropouts, lr, weight_decay): #hidden_dims,dropouts为俩列表
        super().__init__()
        self.save_hyperparameters() #自动保存__init__方法中的所有输入参数到self.hparams中
        layers = []
        in_dim = input_dim
        for i, hidden_dim in enumerate(hidden_dims):
            layers.append(nn.BatchNorm1d(in_dim))
            if i > 0:
                layers.append(nn.SiLU())
            if i < len(dropouts):
                layers.append(nn.Dropout(dropouts[i]))
            layers.append(nn.Linear(in_dim, hidden_dim))
            # layers.append(nn.ReLU())
            in_dim = hidden_dim
        layers.append(nn.Linear(in_dim, 1))  # 输出层
        layers.append(nn.Tanh())
        self.model = nn.Sequential(*layers) #将所有层组合存放至Sequential
        self.lr = lr
        self.weight_decay = weight_decay #L2正则化系数
        self.validation_step_outputs = []

    def forward(self, x):
        return 5 * self.model(x).squeeze(-1)  # 输出为一维张量，输出缩放至[-5,5]

    def training_step(self, batch):
        x, y, w = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y, reduction='none') * w  # 考虑样本权重的加权MSE
        loss = loss.mean()
        self.log('train_loss', loss, on_step=False, on_epoch=True, batch_size=x.size(0))
        return loss

    def validation_step(self, batch):
        x, y, w = batch
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y, reduction='none') * w
        loss = loss.mean()
        self.log('val_loss', loss, on_step=False, on_epoch=True, batch_size=x.size(0))
        self.validation_step_outputs.append((y_hat, y, w))
        return loss

    def on_validation_epoch_end(self):
        """Calculate validation WRMSE at the end of the epoch."""
        y = torch.cat([x[1] for x in self.validation_step_outputs]).cpu().numpy()
        if self.trainer.sanity_checking: #Sanity Check 调试阶段
            prob = torch.cat([x[0] for x in self.validation_step_outputs]).cpu().numpy()
        else:
            prob = torch.cat([x[0] for x in self.validation_step_outputs]).cpu().numpy()
            weights = torch.cat([x[2] for x in self.validation_step_outputs]).cpu().numpy()
            # r2_val
            val_r_square = r2_val(y, prob, weights)
            self.log("val_r_square", val_r_square, prog_bar=True, on_step=False, on_epoch=True)
        self.validation_step_outputs.clear() #清空缓存

    def configure_optimizers(self):
        #动态优化器，根据验证损失调整学习率。当 val_loss 在 5 个 epoch 内未下降时，学习率减半（factir = 0.5)
        optimizer = torch.optim.Adam(self.parameters(), 
                                     lr=self.lr, 
                                     weight_decay=self.weight_decay
        )
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, 
            mode='min', 
            factor=0.5, 
            patience=5,                                                               
            verbose=True
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss', 
            }
        }

    def on_train_epoch_end(self):
        if self.trainer.sanity_checking:
            return
        epoch = self.trainer.current_epoch
        metrics = {k: v.item() if isinstance(v, torch.Tensor) else v for k, v in self.trainer.logged_metrics.items()}
        formatted_metrics = {k: f"{v:.5f}" for k, v in metrics.items()}
        print(f"Epoch {epoch}: {formatted_metrics}")

In [5]:
N_folds = 5
# 加载最佳模型
models = []
for fold in range(N_folds):
    checkpoint_path = f"{CONFIG.model_paths[0]}/nn_{fold}.model"
    model = NN.load_from_checkpoint(checkpoint_path)
    models.append(model.to("cuda:0"))

In [6]:
#valid = pl.scan_parquet(
#    f"/kaggle/input/data-1200-new/train.parquet"
#).filter(pl.col('date_id')>=1650).collect().to_pandas()

In [7]:
#X_valid = valid[ CONFIG.xgb_feature_cols ]
#y_valid = valid[ CONFIG.target_col ]
#w_valid = valid[ "weight" ]
#y_pred_valid_xgb = xgb_model.predict(X_valid)
#valid_score = r2_score( y_valid, y_pred_valid_xgb, sample_weight=w_valid )
#valid_score

In [8]:
#X_valid = valid[ CONFIG.nn_feature_cols ]
#X_valid.rename(columns=CONFIG.rename_cols, inplace=True)
#y_valid = valid[ CONFIG.target_col ]
#w_valid = valid[ "weight" ]
#X_valid = X_valid.fillna(method = 'ffill').fillna(0)
#X_valid.shape, y_valid.shape, w_valid.shape

In [9]:
#y_pred_valid_nn = np.zeros(y_valid.shape)
#with torch.no_grad():
#    for model in models:
#        model.eval()
#        y_pred_valid_nn += model(torch.FloatTensor(X_valid.values).to("cuda:0")).cpu().numpy() / len(models)
#valid_score = r2_score( y_valid, y_pred_valid_nn, sample_weight=w_valid )
#valid_score

In [10]:
#y_pred_valid_ensemble =  (y_pred_valid_xgb*1/3 + y_pred_valid_nn*2/3)
#valid_score = r2_score( y_valid, y_pred_valid_ensemble, sample_weight=w_valid )
#valid_score

In [11]:
#y_pred_valid_ensemble =  (y_pred_valid_xgb*2/4 + y_pred_valid_nn*2/4)
#valid_score = r2_score( y_valid, y_pred_valid_ensemble, sample_weight=w_valid )
#valid_score

In [12]:
#y_pred_valid_ensemble =  (y_pred_valid_xgb*2/3 + y_pred_valid_nn*1/3)
#valid_score = r2_score( y_valid, y_pred_valid_ensemble, sample_weight=w_valid )
#valid_score

In [13]:
#del valid, X_valid, y_valid, w_valid
#gc.collect()

In [14]:
history = pl.scan_parquet(
    "/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet"
).select(['date_id','time_id','symbol_id','weight'] + [f"feature_{idx:02d}" for idx in range(79)] + [f"responder_{idx}" for idx in range(9)]).filter(
    (pl.col("date_id")==1697)
).with_columns(
    date_id = (pl.col("date_id") - pl.lit(1698)).cast(pl.Int16)
).collect()

In [15]:
history_column_types = {
    'date_id': pl.Int16,
    'time_id': pl.Int16,
    'symbol_id': pl.Int16,
}

feature_column_types = {}
for f in [f"feature_{idx:02d}" for idx in range(79)]:
    feature_column_types[f] = pl.Float32

responder_column_types = {}
for f in [f"responder_{idx}" for idx in range(9)]:
    responder_column_types[f] = pl.Float32

responder_lag_column_types = {}
for f in [f"responder_{idx}_lag_1" for idx in range(9)]:
    responder_lag_column_types[f] = pl.Float32

history = history.cast(feature_column_types)
history = history.cast(responder_column_types)
history = history.cast(history_column_types)

In [16]:
def create_agg_list(columns):
    agg_mean_list = [pl.col(c).mean().name.suffix(f"_mean_lag") for c in columns]
    agg_std_list = [pl.col(c).std().name.suffix(f"_std_lag") for c in columns]
    agg_max_list = [pl.col(c).max().name.suffix(f"_max_lag") for c in columns]
    agg_first_list = [pl.col(c).first().name.suffix(f"_first_lag") for c in columns]
    agg_last_list = [pl.col(c).last().name.suffix(f"_last_lag") for c in columns]
    agg_chg_list = [(pl.col(c).last()/pl.col(c).first() - 1).name.suffix(f"_chg_lag") for c in columns]
    agg_list = agg_mean_list + agg_std_list + agg_max_list + agg_last_list + agg_chg_list
    return agg_list

In [ ]:
def predict(test: pl.DataFrame, lags: pl.DataFrame | None) -> pl.DataFrame | pd.DataFrame:

    global history
    global lags_infer
    global lags_c
    
    #获取当前的symbol列表，date和time
    symbol_ids = test.select("symbol_id").to_numpy()[:, 0]
    current_date = test.select("date_id").to_numpy()[:, 0][0]
    current_time = test.select('time_id').to_numpy()[:, 0][0]

    #确保数据格式，以便后续相关操作不会出错
    test = test.cast(history_column_types)
    test = test.cast(feature_column_types)

    #初始化DataFrame
    predictions = test.select(
        'row_id',
        pl.lit(0.0).alias('responder_6'),
    )

    #特征工程
    agg_list = create_agg_list(CONFIG.responder_cols) 

    #在每天的第一个时间段
    if current_time == 0 :
        shift_n_data = history.filter(pl.col('date_id') == current_date - 1 ) #取出昨天的所有数据
        lags_infer = shift_n_data.group_by(["date_id", "symbol_id"], maintain_order=True).agg(agg_list) #计算今天要用的lag特征
        lags_infer = lags_infer.with_columns(date_id = (pl.col("date_id") + pl.lit(1)).cast(pl.Int16)) #之后特征date+1对齐当前时间
        history = history.filter(pl.col("date_id") == current_date) #除去昨天的数据

    # 预测时，某个时间段的lags可能会传入一个空集。
    # 如果传入的是空集，则用昨日的lags来进行预测；如果并非空集，则进行更新    
    if lags is not None :
        lags_c = lags
        lags_c = lags_c.rename(CONFIG.lag_cols_rename) 
        lags_c = lags_c.cast(history_column_types)
        lags_c = lags_c.cast(responder_column_types)

    # 更新数据    
    current_data = test.join(lags_c, on=['date_id','time_id','symbol_id'], how='left').drop('row_id','is_scored')
    history = pl.concat([history,current_data]) #拼接，用于更新history
    X_test = test.join(lags_infer, on=["date_id", "symbol_id"], how="left")    

    preds = np.zeros((X_test.shape[0],))
    preds += xgb_model.predict(X_test[CONFIG.xgb_feature_cols].to_pandas()) / 2 #XGB预测，权重0.5

    test_input = X_test[CONFIG.nn_feature_cols].to_pandas()
    test_input = test_input.fillna(method='ffill').fillna(0)
    test_input.rename(columns=CONFIG.rename_cols, inplace=True)
    test_input = torch.FloatTensor(test_input.values).to("cuda:0")
    
    
    with torch.no_grad():
        for i, nn_model in enumerate(tqdm(models)):
            nn_model.eval()
            preds += nn_model(test_input).cpu().numpy() / (len(models)*2) #NN预测，权重0.5

    #Output
    predictions = (
        test.select('row_id').with_columns(
            pl.Series(name='responder_6', values=preds, dtype=pl.Float32)
        )
    )

    assert isinstance(predictions, pl.DataFrame | pd.DataFrame)
    assert list(predictions.columns) == ['row_id', 'responder_6']
    assert len(predictions) == len(test)

    return predictions

In [18]:
inference_server = kaggle_evaluation.jane_street_inference_server.JSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/jane-street-real-time-market-data-forecasting/test.parquet',
            '/kaggle/input/jane-street-real-time-market-data-forecasting/lags.parquet',
        )
    )

  0%|          | 0/5 [00:00<?, ?it/s]